In [1]:
# CNNで MNIST を予測

In [37]:
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.datasets import fetch_mldata

import keras as ks
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import np_utils


In [2]:
# MNIST
mnist = fetch_mldata('MNIST original', data_home='./dataset/')

C:\Users\owner\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function fetch_mldata is deprecated; fetch_mldata was deprecated in version 0.20 and will be removed in version 0.22
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\owner\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function mldata_filename is deprecated; mldata_filename was deprecated in version 0.20 and will be removed in version 0.22
  warnings.warn(msg, category=DeprecationWarning)


In [3]:
type(mnist)

sklearn.utils.Bunch

In [4]:
# 特徴量とターゲットに分割
X, y = mnist['data'], mnist['target']

In [20]:
# 訓練データとテストデータに分割
X_train, X_test = train_test_split(X, test_size=.2, random_state=42)
y_train, y_test = train_test_split(y, test_size=.2, random_state=42)

In [21]:
X_train.shape, X_test.shape

((56000, 784), (14000, 784))

In [22]:
# 訓練データとテストデータを 10,000件に縮小
X_train = X_train[:10000]
y_train = y_train[:10000]

X_test = X_test[:10000]
y_test = y_test[:10000]

In [23]:
X_train.shape, X_test.shape

((10000, 784), (10000, 784))

In [24]:
y_train.shape, y_test.shape

((10000,), (10000,))

In [25]:
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

In [26]:
# データを整形 channels_last = (batch, height, width, channels)

In [27]:
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

In [28]:
X_train.shape, X_test.shape

((10000, 28, 28, 1), (10000, 28, 28, 1))

In [29]:
y_train[:5], y_test[:5]

(array([7., 7., 1., 5., 0.]), array([7., 8., 6., 5., 7.]))

In [30]:
# ターゲットをクラスベクトルに変換
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

In [32]:
y_train.shape, y_test.shape

((10000, 10), (10000, 10))

In [33]:
y_train[:5], y_test[:5]

(array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32),
 array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]], dtype=float32))

In [34]:
# model
model = Sequential()

In [38]:
# 入力層と畳み込み層を定義
model.add(Conv2D(8, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(28, 28, 1)))

Instructions for updating:
Colocations handled automatically by placer.


In [39]:
# プール層
model.add(MaxPooling2D(pool_size=(2, 2)))

In [40]:
# ドロップアウト
model.add(Dropout(0.25))

In [41]:
# 全結合層
# １次元にする
model.add(Flatten())
model.add(Dense(128, activation='relu'))

In [42]:
# 出力層
model.add(Dense(10, activation='softmax'))

In [43]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 8)         80        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 8)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 13, 13, 8)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1352)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               173184    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total params: 174,554
Trainable params: 174,554
Non-trainable params: 0
________________________________________________

In [44]:
# モデルコンパイル
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [46]:
# モデル訓練
model.fit(X_train, y_train, epochs=50)

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
10000/10000 [==============================] - 7s 675us/step - loss: 1.5131 - accuracy: 0.5597
Epoch 2/50
10000/10000 [==============================] - 3s 253us/step - loss: 0.5360 - accuracy: 0.8360
Epoch 3/50
10000/10000 [==============================] - 2s 245us/step - loss: 0.4226 - accuracy: 0.8719
Epoch 4/50
10000/10000 [==============================] - 3s 270us/step - loss: 0.3841 - accuracy: 0.8836
Epoch 5/50
10000/10000 [==============================] - 3s 256us/step - loss: 0.3505 - accuracy: 0.8932
Epoch 6/50
10000/10000 [==============================] - 3s 261us/step - loss: 0.3257 - accuracy: 0.9018
Epoch 7/50
10000/10000 [==============================] - 3s 305us/step - loss: 0.2977 - accuracy: 0.9101
Epoch 8/50
10000/10000 [==============================] - 3s 297us/step - loss: 0.2842 - accuracy: 0.9143
Epoch 9/50
10000/10000 [==============================] - 3s 279us/step - loss: 0.2708 - accuracy: 0.918

In [47]:
# テストデータで予測
cnn_pred = model.predict(X_test)

In [56]:
cnn_pred[:5], np.argmax(cnn_pred[:5], axis=1)

(array([[1.32385157e-05, 5.78995428e-11, 1.68265831e-06, 4.33079549e-06,
         5.44580553e-06, 4.00033377e-06, 2.06744910e-09, 9.98056173e-01,
         4.63088185e-07, 1.91476045e-03],
        [1.83566873e-09, 1.03064765e-10, 4.27590066e-07, 7.09948245e-06,
         1.23247560e-07, 5.96633996e-08, 5.71072301e-10, 7.64689076e-11,
         9.99991536e-01, 6.88230443e-07],
        [2.09133010e-04, 1.12205350e-06, 1.11052634e-06, 4.78815831e-09,
         4.67175312e-07, 3.58433812e-03, 9.74371076e-01, 1.63598735e-11,
         2.18327977e-02, 4.08193017e-08],
        [6.75541003e-07, 5.41632971e-07, 9.78686103e-07, 5.75045357e-04,
         3.15542947e-09, 9.98228967e-01, 3.21914884e-10, 2.53244216e-04,
         7.97471555e-04, 1.42991994e-04],
        [9.90507898e-10, 6.41774411e-10, 6.96641946e-05, 6.81870370e-05,
         2.39361374e-12, 4.44341536e-10, 9.65410477e-12, 9.99856949e-01,
         4.37199924e-06, 7.11626171e-07]], dtype=float32),
 array([7, 8, 6, 5, 7], dtype=int64))

In [61]:
np.where(y_test[:5] == 1)

(array([0, 1, 2, 3, 4], dtype=int64), array([7, 8, 6, 5, 7], dtype=int64))

In [63]:
np.argmax(cnn_pred[:5], axis=1)

array([7, 8, 6, 5, 7], dtype=int64)

In [50]:
# 推測結果をクラスラベルに変更
y_pred_CNN = np.argmax(cnn_pred, axis=1)
y_test_CNN = np.argmax(y_test, axis=1)

In [52]:
confusion_matrix(y_test_CNN, y_pred_CNN)

array([[ 956,    0,    2,    1,    2,    2,    4,    1,   11,    3],
       [   0, 1099,    6,    3,    4,    1,    1,    0,    8,    2],
       [   8,    4,  947,    8,    3,    3,    1,    6,   17,    2],
       [   2,    1,    4,  960,    0,   14,    0,    8,   17,    2],
       [   1,    0,    1,    1,  900,    0,    1,    4,    2,   18],
       [   2,    2,    1,   13,    2,  870,    4,    1,   12,   12],
       [   6,    1,    0,    1,    2,    7, 1011,    1,    3,    0],
       [   3,    0,   10,    1,    6,    1,    1, 1033,    3,   11],
       [   5,   10,    3,   12,    1,    9,    4,    2,  934,    6],
       [   8,    2,    0,   14,   15,    4,    1,   11,    9,  889]],
      dtype=int64)

In [53]:
accuracy_score(y_test_CNN, y_pred_CNN)

0.9599